## Pos/Neg txt Read

In [ ]:
# readline_all.py
neg=[]
f = open("neg.txt", 'r')
while True:
    line = f.readline()
    if not line: break
    neg.append(line)
f.close()

# readline_all.py
pos=[]
f = open("pos.txt", 'r')
while True:
    line = f.readline()
    if not line: break
    pos.append(line)
f.close()

In [2]:
def delete_blank(document):
    result=[]
    for i in range(len(document)):
        if i % 2 == 0:
            result.append(document[i])
    return result

pos_Text = delete_blank(pos)
neg_Text = delete_blank(neg)

In [3]:
vailidation_Text = pos_Text+neg_Text

In [4]:
vailidation_result = []
for i in vailidation_Text:
    a = i.split("\n")
    vailidation_result.append(a[0])

In [5]:
import json
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import nltk

In [6]:
with open("reviews_Books_5.json","r") as data_file:
    raw_data=data_file.readlines()

In [7]:
json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        print(raw_data[i],i)

{"reviewerID": "AAXUNK0W2DZG5", "asin": "0060520841", "reviewerName": "Amazon Customer \"leneker\"", "helpful": [5, 10], "reviewText": "1996 Bernard Goldberg wrote an editorial for the Wall Street Journal that said there was an obvious bias on the part of network new shows  for the liberal point of view.  he then illustrated this with an example that he dissected in-depth.  The reaction to this observation was the ruination of her career, and the beginning of his status as a pariah among most newsmen.  This book is used to add more ammo to the controversy.That the journalists who so eagerly pry into other peoples lives and business should be reluctant to be examined is hardly surprising.  Almost no one really wants to have his life or business dissected by Mike Wallace not even Dan Rather.  Some facts in this book are really potent such as the survey results which show how the average journalist and the average American are often vastly at odds.  Other chapters highlight different stor

In [8]:
review_data = []

for i in range(len(json_list)):
    review_dict={}
    review_dict["reviewText"] = json_list[i]["reviewText"]
    review_dict["overall"] = json_list[i]["overall"]
    review_data.append(review_dict)

In [9]:
import random
random_data = random.sample(review_data,len(review_data))

In [10]:
reviewText = []
for i in range(len(random_data)):
    reviewText.append(random_data[i]["reviewText"].lower())

## 텍스트에서 Pos/Neg 단어 추출

In [11]:
filtered_sentence_result = []
join_text = []
from nltk.tokenize import word_tokenize

for i in reviewText:
    word_tokens = word_tokenize(i)
    filtered_sentence = [w for w in word_tokens if w in vailidation_result]
    filtered_sentence_result.append(filtered_sentence)

In [12]:
join_filtered_sentence_result = []
for i in filtered_sentence_result:
    a = " ".join(i)
    join_filtered_sentence_result.append(a)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True, max_df=0.3 ,min_df = 2, lowercase=True, stop_words = 'english')
D = vectorizer.fit_transform(join_filtered_sentence_result)

In [14]:
y_data = []
int_y_data = []
for i in range(len(random_data)):
    y_data.append(random_data[i]["overall"])
y_data=np.array(y_data)

### 비율 맞추기.......?

In [15]:
from sklearn import cross_validation

C:\Users\samsung\Miniconda3\envs\workspace\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [16]:
skf = cross_validation.StratifiedKFold(y_data, n_folds=10,random_state=500)

In [17]:
for train_index, test_index in skf:
    X_train, X_test = D[train_index], D[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]

In [18]:
from collections import Counter
Counter(y_test)

Counter({1.0: 423, 2.0: 510, 3.0: 1017, 4.0: 2155, 5.0: 5211})

In [19]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((83868, 3697), (83868,), (9316, 3697), (9316,))

### Multiclass SVM

In [20]:
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

result = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train, y_train)
result.predict(X_test)

array([ 5.,  3.,  3., ...,  5.,  5.,  5.])

In [21]:
from sklearn.metrics import accuracy_score
accuracy_score(result.predict(X_test),y_test)

0.58018462859596398

### LogisticRegression

In [22]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression()

logreg.fit(X_train, y_train)
logreg.predict(X_test) == y_test

array([False,  True,  True, ...,  True,  True, False], dtype=bool)

In [23]:
from sklearn.metrics import accuracy_score
accuracy_score(logreg.predict(X_test), y_test)

0.58641047659939893

### Neural network

In [25]:
D.shape

(93184, 3697)

In [24]:
D = D.toarray()

In [29]:
y_data = []
int_y_data = []
for i in range(len(random_data)):
    y_data.append(random_data[i]["overall"])
for i in y_data:
    int_y_data.append((int(i),))

In [30]:
y_data = np.array(MultiLabelBinarizer().fit_transform(int_y_data))

In [31]:
import tensorflow as tf

x = tf.placeholder(tf.float32, [93184, 3697])

w1 = tf.Variable(tf.truncated_normal([3697, 300], stddev = 0.1))
b1 = tf.Variable(tf.truncated_normal([300]))
w2 = tf.Variable(tf.truncated_normal([300, 5], stddev = 0.1))
b2 = tf.Variable(tf.truncated_normal([5]))

output =  tf.sigmoid(tf.matmul(x, w1) + b1)
hypothesis = tf.sigmoid(tf.matmul(output,w2)+b2)

cost = -tf.reduce_sum(y_data * tf.log(tf.clip_by_value(hypothesis,1e-10,1.0)))
train_step = tf.train.AdamOptimizer(0.01).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(200):
    sess.run(train_step, feed_dict={x:D})
    if i % 100 == 0:
        print(i, sess.run(cost, feed_dict={x:D}))
        
# 뉴럴넷
# 0.1 , 0.01 49%
# 0.1 , 0.005 53%
# 300 : 2034, 53%
# 300 : 60901 ??

0 22277.7
100 0.119099


In [32]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y_data,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print(sess.run(accuracy, feed_dict={x: D}))
# 정확도 측정

0.0549021
